<a href="https://colab.research.google.com/github/AnuragBhattacharjee17/CoDesk/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import numpy
import tflearn
import tensorflow as tf
from tensorflow import reset_default_graph
import random
import pickle

In [0]:
!wget https://raw.githubusercontent.com/AnuragBhattacharjee17/CoDesk/master/train.json

--2019-11-28 08:28:29--  https://raw.githubusercontent.com/AnuragBhattacharjee17/CoDesk/master/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1528 (1.5K) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]   1.49K  --.-KB/s    in 0s      

2019-11-28 08:28:29 (319 MB/s) - ‘train.json’ saved [1528/1528]



In [0]:
import json
with open ('/content/train.json') as file:
  data = json.load(file)

In [0]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1


    training.append(bag)
    output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)



In [0]:
reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 3999  | total loss: 0.38053 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.38053 - acc: 0.9761 -- iter: 24/26
Training Step: 4000  | total loss: 0.36769 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.36769 - acc: 0.9785 -- iter: 26/26
--


In [0]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: Hello
Good to see you again!
You: How are you
Good to see you again!
You: How are you ?
Hi there, how can I help?
You: How r u ?
I am good
You: How old are u ?
22 years young!
You: Bye
Good to see you again!
You: GoodBye
Sad to see you go :(
You: quit
